# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np

In [ ]:
#| export
from infepy.preprocessing import read_landmarks, read_nodes
from infepy.utils import read_toml

In [ ]:
#| export
def morphing(source_template: np.ndarray, # coordinates of the template mesh 
             source_landmarks: np.ndarray,  # Landmarks of the source template
             target_landmarks: np.ndarray,  # Target Landmarks
             ):
    "Morph the target geometry with RBF function - Thin Plate Spine. Plot the mesh and morphed mesh."
        
        #rbf = RBF(original_control_points=source_landmarks, deformed_control_points=target_landmarks,
        #       func='thin_plate_spline', radius=1.0)
        #return rbf(source_template)
    
    pass

In [ ]:
def iter ():
    "This function checks if there are multiple target to morph."
    # https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right  check out command os.walk https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right
    config = read_toml()
    #try
    # match_file = [item for item in config['target'] if item.endswith("log")] # return a empty list if there are no match
    # if match_file is not []:
    #     print("There are csv files. ")
    # else:
    #     pass
    return

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()